In [1]:
import ast
import numpy as np
import pandas as pd
from config import EXTERNAL_DATA_DIR

2025-02-10 13:15:06.666 | INFO     | config:<module>:11 - PROJ_ROOT path is: /Users/manuelbolz/Documents/git/for_work/company_success_prediction


In [2]:
shab_messages = pd.read_csv(EXTERNAL_DATA_DIR / 'all_shab_messages_parsed.csv')
shab_messages['parsed_variables'] = shab_messages.parsed_variables.apply(ast.literal_eval)
shab_messages.head()

,shab_id,keyword,text_slice,parsed_variables,main_group
0,2636073,BEGINNING,"['Rubigenhof Fischzucht AG , in Rubigen , CHE-...",{},undetermined
1,2636073,domizil neu,"['Alte Belpstrasse 5 , 3113 Rubigen']","{'adressen_neu': [{'care_of': '', 'strasse': '...",firmen und adressaenderungen
2,2636075,BEGINNING,"['Käsereigenossenschaft Rüegsbach , in Rüegsau...",{},undetermined
3,2636075,domizil neu,"['c/o Peter Stalder , Lehn 265 , 3418 Rüegsbach']","{'adressen_neu': [{'care_of': 'Peter Stalder',...",firmen und adressaenderungen
4,2636075,mitteilungen,['Mitteilungen an die Genossenschafter : schri...,{},undetermined


In [3]:
capital_changes = shab_messages[shab_messages.main_group == 'kapital- und rechtsformänderungen'].copy()
mergers_and_acquisitions = shab_messages[shab_messages.main_group == 'fusionen und ab- und aufspaltungen'].copy()

In [4]:
# Correct wrong values like '446.001.000.00'
def correct_number(number_str: str) -> str:
    parts = number_str.rsplit('.', 1)
    return parts[0].replace('.', '') + '.' + parts[1] if len(parts) > 1 else parts[0].replace('.', '')

# PROCESS CAPITAL CHANGES

In [5]:
shab_ids = [[], []]
keywords = [[], []]
main_groups = [[], []]

kapital_neu = []
kapital_bisher = []
anz_scheine_neu = []
wert_scheine_neu = []
typ_scheine_neu = []

for i, row in capital_changes.iterrows():
    shab_id = row['shab_id']
    keyword = row['keyword']
    main_group = row['main_group']

    parsed_variables = row['parsed_variables']
    k_neu = parsed_variables.get('kapital_neu', [])
    k_bisher = parsed_variables.get('kapital_bisher', [])
    s_neu = parsed_variables.get('scheine_neu', [])

    if k_neu or k_bisher:
        kapital_neu.append(k_neu[0] if len(k_neu) > 0 else None)
        kapital_bisher.append(k_bisher[0] if len(k_bisher) > 0 else None)
        shab_ids[0].append(shab_id)
        keywords[0].append(keyword)
        main_groups[0].append(main_group)
    if s_neu:
        for s in s_neu:
            anz_scheine_neu.append(s.get('anzahl'))
            wert_scheine_neu.append(s.get('wert'))
            typ_scheine_neu.append(s.get('typ'))
            shab_ids[1].append(shab_id)
            keywords[1].append(keyword)
            main_groups[1].append(main_group)

In [9]:
cap_new = pd.DataFrame({
    'shab_id': shab_ids[0],
    'keyword': keywords[0],
    'hauptkategorie': main_groups[0],
    'kapital_neu': kapital_neu,
    'kapital_bisher': kapital_bisher})

cap_new['kapital_neu'] = cap_new['kapital_neu'].fillna('').str.replace("'", "", regex=False)
cap_new['kapital_bisher'] = cap_new['kapital_bisher'].fillna('').str.replace("'", "", regex=False)

# Extract Währung
cap_new['waehrung_neu'] = cap_new['kapital_neu'].str.extract(r'^([^\d\s]+)')
cap_new['waehrung_neu'] = cap_new['waehrung_neu'].fillna('')

cap_new['waehrung_bisher'] = cap_new['kapital_bisher'].str.extract(r'^([^\d\s]+)')
cap_new['waehrung_bisher'] = cap_new['waehrung_bisher'].fillna('')

# Extract Kapital
cap_new['kapital_neu'] = cap_new['kapital_neu'].str.extract(r'([\d.,]+)').astype(str)
cap_new['kapital_bisher'] = cap_new['kapital_bisher'].str.extract(r'([\d.,]+)').astype(str)

# Apply correction to remove unneccessary punctuations
cap_new['kapital_neu'] = cap_new['kapital_neu'].apply(correct_number)
cap_new['kapital_bisher'] = cap_new['kapital_bisher'].apply(correct_number)

cap_new['kapital_neu'] = [v.replace(',', '.') if not '.' in v else v.replace(',', '') for v in cap_new['kapital_neu']]
cap_new.loc[cap_new['kapital_neu'] == '.', 'kapital_neu'] = np.nan

cap_new['kapital_bisher'] = [v.replace(',', '.') if not '.' in v else v.replace(',', '') for v in cap_new['kapital_bisher']]
cap_new.loc[cap_new['kapital_bisher'] == '.', 'kapital_bisher'] = np.nan

# Ensure correct types
cap_new['kapital_neu'] = cap_new['kapital_neu'].astype(float)
cap_new['kapital_bisher'] = cap_new['kapital_bisher'].astype(float)

In [10]:
stocks_new = pd.DataFrame({
    'shab_id': shab_ids[1],
    'keyword': keywords[1],
    'hauptkategorie': main_groups[1],
    'anz_scheine_neu': anz_scheine_neu,
    'wert_scheine_neu': wert_scheine_neu,
    'typ_scheine_neu': typ_scheine_neu,})

stocks_new['wert_scheine_neu'] = stocks_new['wert_scheine_neu'].fillna('').str.replace("'", "", regex=False)

# Extract Währung
stocks_new['waehrung_scheine_neu'] = stocks_new['wert_scheine_neu'].str.extract(r'^([^\d\s]+)')
stocks_new['waehrung_scheine_neu'] = stocks_new['waehrung_scheine_neu'].fillna('')

# Extract number of Stocks, etc.
stocks_new['wert_scheine_neu'] = stocks_new['wert_scheine_neu'].str.extract(r'([\d.,]+)').astype(str)

# Apply correction to remove unneccessary punctuations
stocks_new['wert_scheine_neu'] = stocks_new['wert_scheine_neu'].apply(correct_number)
stocks_new['wert_scheine_neu'] = [v.replace(',', '.') if not '.' in v else v.replace(',', '') for v in stocks_new['wert_scheine_neu']]
stocks_new.loc[stocks_new['wert_scheine_neu'] == '.', 'wert_scheine_neu'] = np.nan

# Ensure correct types
stocks_new['wert_scheine_neu'] = stocks_new['wert_scheine_neu'].astype(float)
stocks_new['anz_scheine_neu'] = stocks_new['anz_scheine_neu'].astype(int)

In [11]:
# Calculate total value of the capital by multiplying the number of shares with their individual value
stocks_new['wert_total'] = stocks_new['anz_scheine_neu'] * stocks_new['wert_scheine_neu']

# Calculate new capital for shab ids where kapital_neu variable is not given, but scheine_neu is
missing_ids = set(stocks_new.shab_id).difference(set(cap_new.shab_id))
stocks_new_missing = stocks_new[stocks_new.shab_id.isin(missing_ids)]

cap_new_missing = stocks_new_missing.groupby(['shab_id', 'keyword', 'hauptkategorie']).agg(
    kapital_neu=pd.NamedAgg(column='wert_total', aggfunc='sum'),
    waehrung_neu=pd.NamedAgg(column='waehrung_scheine_neu', aggfunc=lambda x: list(set([currency for currency in x if currency != ''])))).reset_index()

In [12]:
# Check if there are mixed currencies
assert len([cur_set for cur_set in cap_new_missing['waehrung_neu'] if len(cur_set) > 1]) == 0

cap_new_missing['waehrung_neu'] = [v[0] if len(v) > 0 else '' for v in cap_new_missing['waehrung_neu']]
cap_new_missing['waehrung_bisher'] = ''
cap_new_missing['kapital_bisher'] = np.nan

In [13]:
# Combine the two dataframes to get all capital changes
cap_new_concat = pd.concat([cap_new, cap_new_missing])

In [14]:
cap_new_concat.head()

,shab_id,keyword,hauptkategorie,kapital_neu,kapital_bisher,waehrung_neu,waehrung_bisher
0,2636095,aktienkapital neu,kapital- und rechtsformänderungen,212109.31,170500.53,CHF,CHF
1,2636095,liberierung aktienkapital neu,kapital- und rechtsformänderungen,212109.31,170500.53,CHF,CHF
2,2636125,stammkapital neu,kapital- und rechtsformänderungen,28000.00,21000.00,CHF,CHF
3,2636131,stammkapital neu,kapital- und rechtsformänderungen,20000.00,NaN,CHF,
4,2636147,liberierung aktienkapital neu,kapital- und rechtsformänderungen,100000.00,NaN,CHF,


In [15]:
cap_new_concat.to_csv(EXTERNAL_DATA_DIR / 'capital_changes.csv', index=False)

# PROCESS MERGERS AND ACQUISIITONS

In [16]:
mergers_and_acquisitions

,shab_id,keyword,text_slice,parsed_variables,main_group
12449,2658475,fusion,"[""Übernahme der Aktiven und Passiven der Uniwo...","{'vertragsdatum': ['04.02.2016'], 'bilanzdatum...",fusionen und ab- und aufspaltungen
12984,2659919,fusion,"[""Übernahme der Aktiven und Passiven der Guten...","{'vertragsdatum': ['26.01.2016'], 'bilanzdatum...",fusionen und ab- und aufspaltungen
16591,2664053,fusion,"[""Übernahme der Aktiven und Passiven der Acher...","{'vertragsdatum': ['08.02.2016'], 'bilanzdatum...",fusionen und ab- und aufspaltungen
16677,2664091,fusion,"[""Übernahme der Aktiven und Passiven der Centr...","{'vertragsdatum': ['26.01.2016'], 'bilanzdatum...",fusionen und ab- und aufspaltungen
17363,2665907,fusion,"[""Übernahme der Aktiven und Passiven der Fores...","{'vertragsdatum': ['02.02.2016'], 'bilanzdatum...",fusionen und ab- und aufspaltungen
...,...,...,...,...,...
5206707,1006233808,fusion,"[""Übernahme der Aktiven und Passiven der BYTEC...","{'vertragsdatum': ['19.12.2024'], 'bilanzdatum...",fusionen und ab- und aufspaltungen
5207599,1006234146,abspaltung,"[""Ein Teil der Aktiven und Passiven geht gemäs...",{},fusionen und ab- und aufspaltungen
5207607,1006234147,abspaltung,"[""Die Gesellschaft entsteht aus der Abspaltung...",{},fusionen und ab- und aufspaltungen
5211975,1006236266,fusion,"[""Übernahme der Aktiven und Passiven der GBSB ...","{'vertragsdatum': ['26.11.2024'], 'bilanzdatum...",fusionen und ab- und aufspaltungen


In [17]:
shab_ids = []
keywords = []
main_groups = []
text_passages = []

vertragsdaten = []
bilanzdaten = []

firma_uebernommen = []
ort_uebernommen = []
id_uebernommen = []
aktiven_uebernommen = []
passiven_uebernommen = []

for i, row in mergers_and_acquisitions.iterrows():
    shab_id = row['shab_id']
    keyword = row['keyword']
    main_group = row['main_group']
    text_passage = row['text_slice']

    parsed_variables = row['parsed_variables']

    if parsed_variables:
        vertragsdatum = parsed_variables.get('vertragsdatum', [])
        bilanzdatum = parsed_variables.get('bilanzdatum', [])
        firmen_uebernommen = parsed_variables.get('firmen_uebernommen', [])
        for firm in firmen_uebernommen:
            firmenname = firm.get('firmenname', '')
            ort = firm.get('ort', '')
            id = firm.get('id', '')
            kapital_uebernommen = firm.get('kapital_uebernommen', {})
            aktiven = kapital_uebernommen.get('aktiven', '') if kapital_uebernommen else ''
            passiven = kapital_uebernommen.get('passiven', '') if kapital_uebernommen else ''

            vertragsdaten.append(vertragsdatum[0] if len(vertragsdatum) > 0 else '')
            bilanzdaten.append(bilanzdatum[0] if len(bilanzdatum) > 0 else '')
            firma_uebernommen.append(firmenname)
            ort_uebernommen.append(ort)
            id_uebernommen.append(id)
            aktiven_uebernommen.append(aktiven)
            passiven_uebernommen.append(aktiven)
            shab_ids.append(shab_id)
            keywords.append(keyword)
            main_groups.append(main_group)
            text_passages.append(text_passage)
    else:
        firma_uebernommen.append('')
        ort_uebernommen.append('')
        id_uebernommen.append('')
        aktiven_uebernommen.append('')
        passiven_uebernommen.append('')
        shab_ids.append(shab_id)
        keywords.append(keyword)
        main_groups.append(main_group)
        text_passages.append(text_passage)

In [18]:
assert len(shab_ids) == len(keywords) == len(main_groups) == len(firma_uebernommen) == len(ort_uebernommen) == len(id_uebernommen) == len(aktiven_uebernommen) == len(passiven_uebernommen)

In [19]:
processed_mergers = pd.DataFrame({
    'shab_id': shab_ids,
    'keyword': keywords,
    'hauptkategorie': main_groups,
    'texte': text_passages,
    'firma_uebernommen': firma_uebernommen,
    'ort_uebernommen': ort_uebernommen,
    'id_uebernommen': id_uebernommen,
    'aktiven_uebernommen': aktiven_uebernommen,
    'passiven_uebernommen': passiven_uebernommen
})

processed_mergers['aktiven_uebernommen'] = processed_mergers['aktiven_uebernommen'].fillna('').str.replace("'", "", regex=False)
processed_mergers['passiven_uebernommen'] = processed_mergers['passiven_uebernommen'].fillna('').str.replace("'", "", regex=False)

# Extract Währung of Aktiven/Passiven
processed_mergers['waehrung_aktiven_uebernommen'] = processed_mergers['aktiven_uebernommen'].str.extract(r'^([^\d\s]+)')
processed_mergers['waehrung_aktiven_uebernommen'] = processed_mergers['waehrung_aktiven_uebernommen'].fillna('')

processed_mergers['waehrung_passiven_uebernommen'] = processed_mergers['passiven_uebernommen'].str.extract(r'^([^\d\s]+)')
processed_mergers['waehrung_passiven_uebernommen'] = processed_mergers['waehrung_passiven_uebernommen'].fillna('')

# Extract value of Aktiven/Passiven
processed_mergers['aktiven_uebernommen'] = processed_mergers['aktiven_uebernommen'].str.extract(r'([\d.,]+)').astype(str)

processed_mergers['passiven_uebernommen'] = processed_mergers['passiven_uebernommen'].str.extract(r'([\d.,]+)').astype(str)

# Apply correction to remove unneccessary punctuations
processed_mergers['aktiven_uebernommen'] = processed_mergers['aktiven_uebernommen'].apply(correct_number)
processed_mergers['aktiven_uebernommen'] = [v.replace(',', '.') if not '.' in v else v.replace(',', '') for v in processed_mergers['aktiven_uebernommen']]

processed_mergers['passiven_uebernommen'] = processed_mergers['passiven_uebernommen'].apply(correct_number)
processed_mergers['passiven_uebernommen'] = [v.replace(',', '.') if not '.' in v else v.replace(',', '') for v in processed_mergers['passiven_uebernommen']]

# Ensure correct types
processed_mergers['aktiven_uebernommen'] = processed_mergers['aktiven_uebernommen'].astype(float)
processed_mergers['passiven_uebernommen'] = processed_mergers['passiven_uebernommen'].astype(float)

In [20]:
processed_mergers.head()

,shab_id,keyword,hauptkategorie,texte,firma_uebernommen,ort_uebernommen,id_uebernommen,aktiven_uebernommen,passiven_uebernommen,waehrung_aktiven_uebernommen,waehrung_passiven_uebernommen
0,2658475,fusion,fusionen und ab- und aufspaltungen,"[""Übernahme der Aktiven und Passiven der Uniwo...",Uniwork AG,Zürich,CHE-105.866.881,21244.77,21244.77,CHF,CHF
1,2659919,fusion,fusionen und ab- und aufspaltungen,"[""Übernahme der Aktiven und Passiven der Guten...",Gutenberg Management AG,Zürich,CHE-106.291.629,1193198.00,1193198.00,CHF,CHF
2,2664053,fusion,fusionen und ab- und aufspaltungen,"[""Übernahme der Aktiven und Passiven der Acher...",Achermann Autospenglerei GmbH,Buochs,CHE-106.902.657,858140.47,858140.47,CHF,CHF
3,2664091,fusion,fusionen und ab- und aufspaltungen,"[""Übernahme der Aktiven und Passiven der Centr...",Centralway Ventures AG,Zug,CHE-243.603.136,877170.91,877170.91,CHF,CHF
4,2665907,fusion,fusionen und ab- und aufspaltungen,"[""Übernahme der Aktiven und Passiven der Fores...",Forest Laboratories Switzerland GmbH,Regensdorf,CHE-266.626.421,205520.00,205520.00,CHF,CHF


In [21]:
processed_mergers.to_csv(EXTERNAL_DATA_DIR / 'merger_sizes.csv', index=False)